In [ ]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer --branch RVE-2.1.0

In [ ]:
# https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ is where all model files for RVE are stored
# you can download them from there, or try custom models from https://openmodeldb.info/
# Use .pth models for upscaling, and rife .pkl for interpolation
!mkdir real-video-enhancer/models/
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.26.pkl
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.22.pkl
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.22-lite.pkl # this is the fastest model
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/2x_ModernSpanimationV2.pth


In [ ]:
#!pip install --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu124 --no-deps torch_tensorrt==2.6.0.dev20241023
#!pip install --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu124 -r real-video-enhancer/backend/requirements.txt --force
!cd real-video-enhancer && mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ffmpeg && chmod +x ffmpeg



```
usage: rve-backend.py [-h] [-i INPUT] [-o OUTPUT] [-l OVERLAP] [-b BACKEND] [--upscaleModel UPSCALEMODEL] [--interpolateModel INTERPOLATEMODEL] [--interpolateFactor INTERPOLATEFACTOR] [--precision PRECISION]
                      [--tensorrt_opt_profile TENSORRT_OPT_PROFILE] [--sceneDetectMethod SCENEDETECTMETHOD] [--sceneDetectSensitivity SCENEDETECTSENSITIVITY] [--overwrite] [--crf CRF]
                      [--custom_encoder CUSTOM_ENCODER] [--tilesize TILESIZE]

Backend to RVE, used to upscale and interpolate videos

options:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        input video path
  -o OUTPUT, --output OUTPUT
                        output video path or PIPE
  -l OVERLAP, --overlap OVERLAP
                        overlap size on tiled rendering (default=10)
  -b BACKEND, --backend BACKEND
                        backend used to upscale image. (pytorch/ncnn/tensorrt/directml, default=pytorch)
  --upscaleModel UPSCALEMODEL
                        Direct path to upscaling model, will automatically upscale if model is valid.
  --interpolateModel INTERPOLATEMODEL
                        Direct path to interpolation model, will automatically upscale if model is valid.
  --interpolateFactor INTERPOLATEFACTOR
                        Multiplier for interpolation, will round up to nearest integer for interpolation but the fps will be correct
  --precision PRECISION
                        sets precision for model, (auto/float16/float32, default=auto)
  --tensorrt_opt_profile TENSORRT_OPT_PROFILE
                        sets tensorrt optimization profile for model, (1/2/3/4/5, default=3)
  --sceneDetectMethod SCENEDETECTMETHOD
                        Scene change detection to avoid interpolating transitions. (options=mean, mean_segmented, none) Mean segmented splits up an image, and if an arbitrary number of segments changes are
                        detected within the segments, it will trigger a scene change. (lower sensativity thresholds are not recommended)
  --sceneDetectSensitivity SCENEDETECTSENSITIVITY
                        Scene change detection sensitivity, lower number means it has a higher chance of detecting scene changes, with risk of detecting too many.
  --overwrite           Overwrite output video if it already exists.
  --crf CRF             Constant rate factor for videos, lower setting means higher quality.
  --custom_encoder CUSTOM_ENCODER
                        custom encoder
  --tilesize TILESIZE   upscale images in smaller chunks, default is the size of the input video
```



In [ ]:
#example upscale/denoise command
!cd real-video-enhancer/ && python3 backend/rve-backend.py -i ../{INPUT_FILE} -o ../{OUTPUT_FILE} --upscaleModel models/{MODEL} --backend pytorch

In [ ]:
# example interpoalte command
!cd real-video-enhancer/ && python3 backend/rve-backend.py -i ../{INPUT_FILE} -o ../{OUTPUT_FILE} --interpolateModel models/rife4.26.pkl --backend pytorch --interpolateFactor 2

In [ ]:
!cd real-video-enhancer/ && cat backend_log.txt